# Automated Classification of IT Service Tickets Using OpenAI API

---
**Project Objective**: Leverage the OpenAI API to automatically classify IT service tickets into predefined categories, including:


1. Hardware
2.  Acces
3. 3 Miscellaneo
4. 
4 HR Supp
5. 
5 Purc
6. e
6 Administrative r
7. ts
7 S
8. age
8 Internal   

Post-classification, the results will be compared with pre-labeled categories to assess accuracy and effectiveness.Project

## Installing Necessary Library and Dependency

In [28]:
#pip uninstall -y openai pydantic typing_extensions
#start cost $3.69


^C
Note: you may need to restart the kernel to use updated packages.
Found existing installation: openai 1.55.3
Uninstalling openai-1.55.3:
  Successfully uninstalled openai-1.55.3
Found existing installation: pydantic 1.10.9
Uninstalling pydantic-1.10.9:
  Successfully uninstalled pydantic-1.10.9
Found existing installation: typing_extensions 4.11.0
Uninstalling typing_extensions-4.11.0:
  Successfully uninstalled typing_extensions-4.11.0


You can safely remove it manually.


In [1]:
pip install openai==1.55.3 pydantic==1.10.9 typing_extensions==4.11


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.0 which is incompatible.



  Using cached openai-1.55.3-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-1.10.9-cp311-cp311-win_amd64.whl.metadata (149 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
Using cached openai-1.55.3-py3-none-any.whl (389 kB)
Using cached pydantic-1.10.9-cp311-cp311-win_amd64.whl (2.1 MB)
Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)


In [2]:
# Installing Kaggle in this environment
!pip install kaggle

# To ignore warinings

import warnings
warnings.filterwarnings('ignore')

In [13]:
#importing libraries and Dependencies
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
from kaggle.api.kaggle_api_extended import KaggleApi



**Authentication**
---

The dataset used in this project was gotten from Kaggle ([Dataset Link](https://www.kaggle.com/datasets/adisongoh/it-service-ticket-classification-dataset")). 

To automate workflows and safeguard API keys in our project, we securely store credentials in an Excel file hosted on OneDrive. This approach allows for seamless integration and centralized management of sensitive information.


In [15]:
#importing OpenAI credentials and Kaggle API stored as excel file
credentials = pd.read_excel("OneDrive/ALT School/kaggle_open.xlsx")

In [21]:
# Setting up Kaggle username and API key
import os
os.chdir("../")
os.environ["KAGGLE_USERNAME"] = credentials["Username"].iloc[0]
os.environ["KAGGLE_KEY"] = credentials["API Keys"].iloc[0]
api= KaggleApi()
api.authenticate()

In [23]:
#Specifying the dataset to download from Kaggle
dataset_name = "adisongoh/it-service-ticket-classification-dataset"
#Setting the path where to save the downloaded data
download_path = Path("OneDrive/ALT School/")

# Create the directory if it doesn't exist
download_path.mkdir(parents=True, exist_ok=True)

# Download the dataset
api.dataset_download_files(dataset_name, path=download_path, unzip=False)

# List the files in the downloaded dataset directory
dataset_files = list(download_path.glob("*"))
print("Files in the dataset directory:", dataset_files)

Dataset URL: https://www.kaggle.com/datasets/adisongoh/it-service-ticket-classification-dataset
Files in the dataset directory: [WindowsPath('OneDrive/ALT School/it-service-ticket-classification-dataset.zip')]


In [25]:
#unzipping file that is downloaded
import zipfile

# Define the path to your ZIP file
zip_path = "OneDrive/ALT School/it-service-ticket-classification-dataset.zip"

# Open the ZIP file and list contents
with zipfile.ZipFile(zip_path, 'r') as z:
    print(z.namelist())  # List files inside the ZIP

# Select the CSV file you want to load (assuming there's only one CSV)
csv_filename = z.namelist()[0]  # Pick the first file

# Read the CSV directly from ZIP
df = pd.read_csv(zip_path, compression="zip")
print(df.head())  # Show first 5 rows


['all_tickets_processed_improved_v3.csv']
                                            Document    Topic_group
0  connection with icon icon dear please setup ic...       Hardware
1  work experience user work experience user hi w...         Access
2  requesting for meeting requesting meeting hi p...       Hardware
3  reset passwords for external accounts re expir...         Access
4  mail verification warning hi has got attached ...  Miscellaneous


In [39]:
#printing first 5 rows of the dataset
df.head()

,text,category
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous


In [27]:
# I forgot how to use the rename function and the help function serve as usefull reminder
help(pd.DataFrame.rename)

Help on function rename in module pandas.core.frame:

rename(self, mapper: 'Renamer | None' = None, *, index: 'Renamer | None' = None, columns: 'Renamer | None' = None, axis: 'Axis | None' = None, copy: 'bool | None' = None, inplace: 'bool' = False, level: 'Level | None' = None, errors: 'IgnoreRaise' = 'ignore') -> 'DataFrame | None'
    Rename columns or index labels.
    
    Function / dict values must be unique (1-to-1). Labels not contained in
    a dict / Series will be left as-is. Extra labels listed don't throw an
    error.
    
    See the :ref:`user guide <basics.rename>` for more.
    
    Parameters
    ----------
    mapper : dict-like or function
        Dict-like or function transformations to apply to
        that axis' values. Use either ``mapper`` and ``axis`` to
        specify the axis to target with ``mapper``, or ``index`` and
        ``columns``.
    index : dict-like or function
        Alternative to specifying axis (``mapper, axis=0``
        is equivalent to

In [28]:
#renaming the columns in the dataframe
df = df.rename(columns={'Document':'text','Topic_group':'category'})

In [29]:
# Checking the Category Column
print('There are {} unique category in this column \nThey are listed below \n'.format(len(df['category'].unique())))
n=0
for cat in df['category'].unique():
    n+=1
    print(n,cat)

There are 8 unique category in this column 
They are listed below 

1 Hardware
2 Access
3 Miscellaneous
4 HR Support
5 Purchase
6 Administrative rights
7 Storage
8 Internal Project


In [32]:
#importing openai library and loading the API key
from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report


client = OpenAI(api_key=credentials["API Keys"].iloc[1])


In [33]:
#creating a function based code snippet in the open AI playground to classify the text
def classify_text(text, categories):
    """Function to classify text into predefined categories using OpenAI API."""
    prompt = f"""
    You are a text classification model. Categorize the following text into one of these categories: {", ".join(categories)}.
    
    Text: "{text}"
    
    Respond only with the category name.
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "developer", "content": prompt}],
        temperature=0  # Ensures consistency
    )

    return response.choices[0].message.content.strip()


In [34]:
# Define categories
categories = ['Hardware', 'Access', 'Miscellaneous', 'HR Support', 'Purchase', 'Administrative rights', 'Storage', 'Internal Project']
#printing categories variable for verification
print(categories)

['Hardware', 'Access', 'Miscellaneous', 'HR Support', 'Purchase', 'Administrative rights', 'Storage', 'Internal Project']


In [35]:
# using only 100 rows of the dataframe to save API cost
df_sample = df.sample(100)

In [40]:
# using lambda function to run our defined function 
df_sample['pred'] = df_sample['text'].apply(lambda x: classify_text(x, categories))

In [37]:
#mapping the category values to numbers
df_sample['pred_num'] = df_sample['pred'].map({ 'Hardware':0, 'Access':1, 'Miscellaneous':2, 'HR Support':3,
                                                             'Purchase':4, 'Administrative rights':5, 'Storage':5, 'Internal Project':6})
df_sample['category_num'] = df_sample['category'].map({ 'Hardware':0, 'Access':1, 'Miscellaneous':2, 'HR Support':3,
                                                             'Purchase':4, 'Administrative rights':5, 'Storage':5, 'Internal Project':6})

In [41]:
#showing top 5 rows
df_sample.head()

,text,category,pred,pred_num,category_num
38704,not connecting a wrong gateway connecting wron...,Hardware,Access,1,0
46002,not working after click report redirecting aga...,Hardware,Access,1,0
17268,new starter nae bucharest thursday march pm he...,HR Support,HR Support,3,3
8125,activation on employee locator activation dear...,Access,Access,1,1
45018,mail is not working sent thursday working hell...,Hardware,Access,1,0


In [38]:
#checking the accuracy of the openAI to the predefined category
print(accuracy_score(df_sample['pred_num'].values, df_sample['category_num'].values))


0.53


### Conclusion
---
The model achieved an accuracy of 53% when compared to the predefined categories. However, this figure should be interpreted cautiously, as the reliability of the predefined categories themselves is uncertain. This uncertainty underscores the challenges in evaluating model performance without a definitive ground truth.

In summary, while the project showcased the advantages of leveraging large language models for text classification without extensive training data, it also illuminated the critical need for dependable evaluation metrics and the potential benefits of domain-specific fine-tuning.
co